<a href="https://colab.research.google.com/github/profshai/pyspark-big-data/blob/main/predict_dog_food_spoiler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predict what causes the dog food to spoil using Random Forest

A dog food company mixes up a batch of preservative that contains 4 different preservative chemicals (A,B,C,D) and then is completed with a "filler" chemical. The food scientists believe one of the A,B,C, or D preservatives is causing the problem.

In this exercise, we want to use Random Forest to find out which parameter had the most predicitive power, thus finding out which chemical causes the early spoiling!

* Pres_A : Percentage of preservative A in the mix
* Pres_B : Percentage of preservative B in the mix
* Pres_C : Percentage of preservative C in the mix
* Pres_D : Percentage of preservative D in the mix
* Spoiled: Label indicating whether or not the dog food batch was spoiled.

In [ ]:
pip install pyspark

     |████████████████████████████████| 212.4MB 70kB/s 
     |████████████████████████████████| 204kB 17.4MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=c8603f6b9bf38ebf768182f7963ca7de75bada1a857919521c1406b359d14ec1
  Stored in directory: /root/.cache/pip/wheels/40/1b/2c/30f43be2627857ab80062bef1527c0128f7b4070b6b2d02139
Successfully built pyspark


In [ ]:
#Tree methods Example
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('dogfood').getOrCreate()

In [ ]:
# Load training data
data = spark.read.csv('dog_food.csv',inferSchema=True,header=True)

In [ ]:
data.printSchema()

root
 |-- A: integer (nullable = true)
 |-- B: integer (nullable = true)
 |-- C: double (nullable = true)
 |-- D: integer (nullable = true)
 |-- Spoiled: double (nullable = true)



In [ ]:
data.head()

Row(A=4, B=2, C=12.0, D=3, Spoiled=1.0)

In [ ]:
data.describe().show()

+-------+------------------+------------------+------------------+------------------+-------------------+
|summary|                 A|                 B|                 C|                 D|            Spoiled|
+-------+------------------+------------------+------------------+------------------+-------------------+
|  count|               490|               490|               490|               490|                490|
|   mean|  5.53469387755102| 5.504081632653061| 9.126530612244897| 5.579591836734694| 0.2857142857142857|
| stddev|2.9515204234399057|2.8537966089662063|2.0555451971054275|2.8548369309982857|0.45221563164613465|
|    min|                 1|                 1|               5.0|                 1|                0.0|
|    max|                10|                10|              14.0|                10|                1.0|
+-------+------------------+------------------+------------------+------------------+-------------------+



In [ ]:
# Import VectorAssembler and Vectors
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [ ]:
data.columns

['A', 'B', 'C', 'D', 'Spoiled']

In [ ]:
assembler = VectorAssembler(inputCols=['A', 'B', 'C', 'D'],outputCol="features")

In [ ]:
output = assembler.transform(data)

In [ ]:
from pyspark.ml.classification import RandomForestClassifier

In [ ]:
rfc = RandomForestClassifier(labelCol='Spoiled',featuresCol='features')

In [ ]:
output.printSchema()

root
 |-- A: integer (nullable = true)
 |-- B: integer (nullable = true)
 |-- C: double (nullable = true)
 |-- D: integer (nullable = true)
 |-- Spoiled: double (nullable = true)
 |-- features: vector (nullable = true)



In [ ]:
final_data = output.select('features','Spoiled')
final_data.head()

Row(features=DenseVector([4.0, 2.0, 12.0, 3.0]), Spoiled=1.0)

In [ ]:
rfc_model = rfc.fit(final_data)

In [ ]:
rfc_model.featureImportances

SparseVector(4, {0: 0.0246, 1: 0.0134, 2: 0.9367, 3: 0.0253})

From the result, The feature at index 2 (Chemical C) is by far the most important feature, meaning it is causing the early spoilage!

End of Notebook!